In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/truth-seekers/sample_submission.csv
/kaggle/input/truth-seekers/train.csv
/kaggle/input/truth-seekers/test.csv


In [2]:
import pandas as pd
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification, AdamW, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [3]:
train_data = pd.read_csv('/kaggle/input/truth-seekers/train.csv')

In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("MYC007/Real-and-Fake-News-Detection")
model = AutoModelForSequenceClassification.from_pretrained("MYC007/Real-and-Fake-News-Detection",
                                                          num_labels = 2)

In [5]:
# tokenizer for BERT
# tokenizer = BertTokenizer.from_pretrained('yinde/fatimah_fake_news_bert', do_lower_case=True)

# custom dataset for PyTorch
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = str(self.data.iloc[idx]['text'])
        label = int(self.data.iloc[idx]['label'])
        inputs = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt',
            truncation=True,
        )
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long),
        }

# DataLoader for training, validation, and test sets
batch_size = 64
train_dataset = CustomDataset(train_data, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# val_dataset = CustomDataset(val_data, tokenizer)
# val_loader = DataLoader(val_dataset, batch_size=batch_size)

# test_dataset = CustomDataset(test_data, tokenizer)
# test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# BERT model for sequence classification
# model = BertForSequenceClassification.from_pretrained('yinde/fatimah_fake_news_bert', num_labels=2)
# model = AutoModelForSequenceClassification.from_pretrained("yinde/fatimah_fake_news_bert",
#                                                           num_labels = 2)

model.to(device)
print('model saved to device')

model saved to device


In [7]:
def training_loop(num_epochs, lr):
    
    # optimizer and learning rate scheduler
    optimizer = AdamW(model.parameters(), lr=lr)
    total_steps = len(train_loader) * num_epochs  
    scheduler = get_linear_schedule_with_warmup(optimizer, 
                                                num_warmup_steps=0, 
                                                num_training_steps=total_steps)

    # loss function
    criterion = nn.CrossEntropyLoss()
    
    # Training loop
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0

        for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}/{num_epochs}'):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            loss.backward()
            optimizer.step()
            scheduler.step()

        avg_train_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch + 1}/{num_epochs} - Average training loss: {avg_train_loss:.4f}")

        # Validation loop
#         model.eval()
#         total_val_loss = 0
#         total_val_correct = 0

#         with torch.no_grad():
#             for batch in tqdm(val_loader, desc=f'Validation'):
#                 input_ids = batch['input_ids'].to(device)
#                 attention_mask = batch['attention_mask'].to(device)
#                 labels = batch['labels'].to(device)

#                 outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#                 loss = outputs.loss
#                 total_val_loss += loss.item()

#                 _, predicted_labels = torch.max(outputs.logits, 1)
#                 total_val_correct += (predicted_labels == labels).sum().item()

#         avg_val_loss = total_val_loss / len(val_loader)
#         val_accuracy = total_val_correct / len(val_data)

#         print(f"Validation loss: {avg_val_loss:.4f} - Validation accuracy: {val_accuracy:.4f}")

In [8]:
training_loop(num_epochs = 4, lr = 2e-5)

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1/4: 100%|██████████| 150/150 [03:34<00:00,  1.43s/it]


Epoch 1/4 - Average training loss: 0.4105


Epoch 2/4: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]


Epoch 2/4 - Average training loss: 0.0388


Epoch 3/4: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]


Epoch 3/4 - Average training loss: 0.0179


Epoch 4/4: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]

Epoch 4/4 - Average training loss: 0.0100


In [9]:
training_loop(num_epochs = 3, lr = 2e-7)

Epoch 1/3: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]


Epoch 1/3 - Average training loss: 0.0081


Epoch 2/3: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]


Epoch 2/3 - Average training loss: 0.0068


Epoch 3/3: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]

Epoch 3/3 - Average training loss: 0.0068


In [10]:
training_loop(num_epochs = 3, lr = 2e-8)

Epoch 1/3: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]


Epoch 1/3 - Average training loss: 0.0075


Epoch 2/3: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]


Epoch 2/3 - Average training loss: 0.0076


Epoch 3/3: 100%|██████████| 150/150 [03:36<00:00,  1.44s/it]

Epoch 3/3 - Average training loss: 0.0072


In [11]:
# save the model
torch.save(model.state_dict(), "myc_model_99pct.pth")

**inference**

In [12]:
test_df = pd.read_csv("/kaggle/input/truth-seekers/test.csv")
test_df['label'] = 1 # adding dummy column
test_df.head(2)

,id,title,text,all,label
0,41786,Ancient Sumerian Writings Reveal The Earth Was...,Ancient Sumerian Writings Reveal The Earth Was...,Ancient Sumerian Writings Reveal The Earth Was...,1
1,37965,Gianno Caldwell Claims Hillary Only Cares Abou...,BREAKING: Trump Set to Break 50-Year-Old Recor...,Gianno Caldwell Claims Hillary Only Cares Abou...,1


In [13]:
infer_dataset = CustomDataset(test_df, tokenizer)
infer_loader = DataLoader(infer_dataset, batch_size=batch_size)

In [14]:
# performance on test loader (different from test csv given)
model.eval()
# total_val_loss = 0
total_val_correct = 0
outputs_list_infer = []
labels_list = []

with torch.no_grad():
    for batch in tqdm(infer_loader, desc=f'test subset'):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        
        # loss = outputs.loss
        # total_val_loss += loss.item()

        _, predicted_labels = torch.max(outputs.logits, 1)
        # total_val_correct += (predicted_labels == labels).sum().item()
        
        outputs_list_infer.append(predicted_labels)
        labels_list.append(labels)

# avg_val_loss = total_val_loss / len(val_loader)
# val_accuracy = total_val_correct / len(val_data)

# print(f"Testing subset accuracy: {val_accuracy:.4f}")

test subset: 100%|██████████| 124/124 [00:54<00:00,  2.27it/s]


In [15]:
# make flat list of outputs and labels
losses_np = [x.cpu().numpy() for x in outputs_list_infer]
flat_list_outputs = [item for sublist in losses_np for item in sublist]

In [16]:
len(flat_list_outputs)

7888

In [17]:
# sample sub
sample_sub = pd.read_csv("/kaggle/input/truth-seekers/sample_submission.csv")
print(sample_sub.shape)
sample_sub['label'] = flat_list_outputs

(7888, 2)


In [18]:
sample_sub = sample_sub.set_index('id')
sample_sub

,label
id,
41786,1
37965,1
55351,1
21861,1
67931,1
...,...
66743,0
65606,0
13423,0


In [19]:
sample_sub.to_csv('pytorch_myc_finetune_finalsub.csv', index = False)

In [20]:
from IPython.display import HTML

sample_sub.to_csv('pytorch_myc_finetune_finalsub.csv')

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='pytorch_myc_finetune_finalsub.csv')